## Crop and combine images

In [ ]:
%cd ../
!git clone https://github.com/junleen/RainNet
%cd RainNet

import os
import torch
from skimage import io
import numpy as np
from util.config import cfg as test_cfg
from data.test_dataset import TestDataset
from util import util
from models.networks import RainNet
from models.normalize import RAIN

%matplotlib inline

In [ ]:
def load_network(cfg):
    net = RainNet(input_nc=cfg.input_nc, 
                output_nc=cfg.output_nc, 
                ngf=cfg.ngf, 
                norm_layer=RAIN, 
                use_dropout=not cfg.no_dropout)
    
    load_path = os.path.join(cfg.checkpoints_dir, cfg.name, 'net_G.pth')
    if not os.path.exists(load_path):
        print('%s not exists. Please check the file'%(load_path))
    print(f'loading the model from {load_path}')
    state_dict = torch.load(load_path)
    util.copy_state_dict(net.state_dict(), state_dict)
    # net.load_state_dict(state_dict)
    return net

def save_img(path, img):
    os.makedirs(os.path.split(path)[0], exist_ok=True)
    io.imsave(path, img)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
rainnet = load_network(test_cfg).cuda()

In [ ]:
base = '/home/cvmlserver/Seohyeon/v-coco/'

padded_folder = os.path.join(base, 'data/padded_masks/')
input_folder = os.path.join(base, 'data/combined/')
final_folder = os.path.join(base, 'data/final/')

bg_names = os.listdir(input_folder)

In [ ]:

comb_path = [base+'data/combined/'+bg_class+'/'+comb for bg_class in bg_names for comb in os.listdir(os.path.join(input_folder, bg_class)) if comb.endswith('.png')]
mask_path = [base+'data/padded_masks/'+bg_class+'/'+file for bg_class in bg_names for file in os.listdir(os.path.join(padded_folder, bg_class))]
print(mask_path[:3])
real_path = comb_path
print(len(comb_path))
print(len(mask_path))

testdata = TestDataset(foreground_paths=comb_path, mask_paths=mask_path, background_paths=real_path, load_size=1024)
print(testdata)
# cv2.imwrite(os.path.join(final_folder, bg_class, image_name), testdata)

In [ ]:
repeat_times = 0 # adjust the foreground image by several times
import cv2
for idx in range(len(testdata)):
    sample = testdata[idx]
    # unsqueeze the data to shape of (1, channel, H, W)
    comp = sample['comp'].unsqueeze(0).to(device)
    mask = sample['mask'].unsqueeze(0).to(device) # if you want to adjust the background to be compatible with the foreground, then add the following command
    # mask = 1 - mask
    real = sample['real'].unsqueeze(0).to(device) # if the real_path is not given, then return composite image by sample['real']
    img_path = sample['img_path']
    pred = rainnet.processImage(comp, mask, real)
    for i in range(repeat_times):
        pred = rainnet.processImage(pred, mask, pred)
        
    # tensor2image
    person_image = cv2.imread(img_path)
    pred_rgb = util.tensor2im(pred[0:1])
    w, h, _ = person_image.shape
    pred_rgb = cv2.resize(pred_rgb, (h, w))
    
    # comp_rgb = util.tensor2im(comp[:1])
    # mask_rgb = util.tensor2im(mask[:1])
    # real_rgb = util.tensor2im(real[:1])
    if not os.path.exists(final_folder+img_path.split('/')[-2]):
        os.mkdir(final_folder+img_path.split('/')[-2])
    print(final_folder+img_path.split('/')[-2]+'/'+img_path.split('/')[-1])
    print(pred_rgb.shape)
    save_img(final_folder+img_path.split('/')[-2]+'/'+img_path.split('/')[-1], pred_rgb)